mount

In [ ]:
from google.colab import drive
from sys import exit
drive.mount('/content/drive')

import os

print(os.listdir())
if "drive" in os.listdir(): os.chdir("drive")
print(os.listdir())
if "MyDrive" in os.listdir(): os.chdir("MyDrive")
print(os.listdir())
if "Colab Notebooks" in os.listdir(): os.chdir("Colab Notebooks")

!pip install pytorch_lightning --quiet

### only save state
# import torch
# from torch import optim, nn, utils, Tensor
# import pytorch_lightning as pl
# import torchvision.models as models
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# from torchmetrics import Accuracy

# model = MLP()
# save_path = "./model_tenhou.pth"
# checkpoint = torch.load(save_path)
# model.load_state_dict(checkpoint['model_state_dict'])
# model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# torch.save(model.state_dict(), "./model_tenhou_less.pth")

learn

In [ ]:

import numpy as np
import torch
from torch import optim, nn, utils, Tensor
import pytorch_lightning as pl
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics import Accuracy

class MLP(pl.LightningModule):

    def __init__(self, obs_size=107*34, n_actions=181, hidden_size=107*34):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
        )

        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=n_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        loss = self.criterion(preds, y)
        acc = self.accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        loss = self.criterion(preds, y)
        acc = self.accuracy(preds, y)
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        loss = self.criterion(preds, y)
        acc = self.accuracy(preds, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer

start_idx = 0 # start file index
file_num = 10 # num of file

##-------------------------------------------------------
def train():
  inps = np.load(f'npy/tenhou_obs{start_idx}.npy')
  for i in range(start_idx, start_idx + file_num):
      print("obs", i)
      inps1 = np.load(f'npy/tenhou_obs{i}.npy')
      inps = np.concatenate([inps, inps1], axis=0)

  tgts = np.load(f'npy/tenhou_actions{start_idx}.npy')
  for i in range(start_idx, start_idx + file_num):
      print("action", i)
      tgts1 = np.load(f'npy/tenhou_actions{i}.npy')
      tgts = np.concatenate([tgts, tgts1])

  dataset = TensorDataset(torch.Tensor(inps),torch.LongTensor(tgts))
  loader = DataLoader(dataset, batch_size=1024)

  ##-------------------------------------------------------
  # val_dataloaders

  # inpsVal = np.load(f'npy/tenhou_obs30.npy')
  # # inpsVal = np.expand_dims(inpsVal, axis=1)
  # tgtsVal = np.load(f'npy/tenhou_actions30.npy')
  # datasetVal = TensorDataset(torch.Tensor(inpsVal),torch.LongTensor(tgtsVal))
  # loaderVal = DataLoader(datasetVal, batch_size=1024)

  ##-------------------------------------------------------

  model = MLP()
  save_path = "./model.pth"
  if os.path.isfile(save_path):
    print('has checkpoint')
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # model.load_state_dict(torch.load('model_tenhou.pth'))
  else:
    print('no checkpoint')
  trainer = pl.Trainer(max_epochs=8)
  # trainer.fit(model=model, train_dataloaders=loader, val_dataloaders=loaderVal)
  trainer.fit(model=model, train_dataloaders=loader)
  print(trainer.callback_metrics)
  torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': model.optimizer.state_dict(),},
            save_path)

  ##-------------------------------------------------------


##-------------------------------------------------------
def test():
  model = MLP()
  save_path = "./model_tenhou.pth"
  if os.path.isfile(save_path):
    print('has checkpoint')
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  else:
    print('no checkpoint')
  start_idx = 0
  file_num = 10
  inpsTest = np.load(f'npy_discard_test/tenhou_obs{start_idx}.npy')
  for i in range(start_idx, start_idx + file_num):
      print("obs", i)
      inps1 = np.load(f'npy_discard_test/tenhou_obs{i}.npy')
      inpsTest = np.concatenate([inpsTest, inps1], axis=0)

  tgtsTest = np.load(f'npy_discard_test/tenhou_actions{start_idx}.npy')
  for i in range(start_idx, start_idx + file_num):
      print("action", i)
      tgts1 = np.load(f'npy_discard_test/tenhou_actions{i}.npy')
      tgtsTest = np.concatenate([tgtsTest, tgts1])

  datasetTest = TensorDataset(torch.Tensor(inpsTest),torch.LongTensor(tgtsTest))
  loaderTest = DataLoader(datasetTest, batch_size=1024)
  trainer = pl.Trainer()
  trainer.test(model, loaderTest)

train()
test()

## disconnect colab

# from google.colab import runtime
# runtime.unassign()